In [1]:
import numpy
import pandas as pd
import csv
import matplotlib.pyplot as plt
import os

In [13]:
#experiment_date='2020-02-17T164323'
#folder_results='results'
#file_prefix='results-'
file_name='traces.csv'

In [53]:
#file=folder_results+'/'+file_prefix+experiment_date+'/'+file_name
cassandra_traces = pd.read_csv(file_name)
tracedata = pd.DataFrame(cassandra_traces)
tracedata.head(5)

,trace_id,span_id,span_hash,duration,flags,logs,operation_name,parent_id,process,refs,start_time,tags,write_time
0,0x00000000000000000123c43201ba2883,82124937279187075,-1723299545951325615,3329,1,"[{ts: 1582792881477698, fields: [{key: 'event'...",calories-parent,0,"{service_name: 'calories', tags: [{key: 'jaege...",NaN,1582792881477682,"[{key: 'sampler.type', value_type: 'string', v...",1582792882175272
1,0x00000000000000000123c43201ba2883,1485135936572115475,7824872964289642957,553,1,NaN,calories-call-0,0,"{service_name: 'calories', tags: [{key: 'jaege...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1582792881477706,"[{key: 'internal.span.format', value_type: 'st...",1582792882175502
2,0x00000000000000000123c43201ba2883,4117494505221816129,8845355937158977620,38,1,NaN,userdata-parent,0,"{service_name: 'userdata', tags: [{key: 'jaege...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1582792881478084,"[{key: 'productIds', value_type: 'string', val...",1582792882175467
3,0x00000000000000000123c43201ba2883,5974327135973288596,-1456599868521519005,388,1,NaN,distances-call-0,0,"{service_name: 'distances', tags: [{key: 'jaeg...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1582792881478392,"[{key: 'internal.span.format', value_type: 'st...",1582792882176156
4,0x00000000000000000123c43201ba2883,7057661615614580298,2178892328650323814,415,1,NaN,distances-parent,0,"{service_name: 'distances', tags: [{key: 'jaeg...","[{ref_type: 'child-of', trace_id: 0x0000000000...",1582792881478382,"[{key: 'legal-basis', value_type: 'string', va...",1582792882176000


1. Identify root traces
1. For each root trace, find children
1. Merge transparency tags as "Sets"
1. Map tags to vocabulary

In [54]:
tracedata.describe()
relevant = tracedata.filter(items=['trace_id','span_id','operation_name', 'refs', 'tags'])
relevant['is_root'] = relevant['refs'].isna()
relevant = relevant.sort_values(by = ['trace_id', 'is_root']).reset_index(drop=True)
relevant.head()
relevant['tags'].head(5)[1]

"[{key: 'categories-storage', value_type: 'string', value_string: '00', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'storage-ttl', value_type: 'string', value_string: '01', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'recipients', value_type: 'string', value_string: '', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'purposes', value_type: 'string', value_string: '00;01;02', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'legal-basis', value_type: 'string', value_string: '01;01;02', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'categories-processing', value_type: 'string', value_string: '00', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'automation', value_type: 'string', value_string: '', value_bool: False, value_long: 0, value_double: 0, value_binary: }, {key: 'productIds', value_type: 'string', value_string: 'rt2'

### Exporting from cassandra with JSON column
As it turns out, cassandra has its own internal model for JSON-formatted columns, which on export to CSV, are not properly formatted JSON (missing double quotes for keys). For SELECT queries in CQL, there is a JSON modifier, that fixes the output for columns, but there is none for CQL's COPY.

- [x] TODO: find a way to parse the column of the CSV or export manually by running SELECT and writing results..?
    - Solution: use regexp parsing (lol)
- [ ] TODO: re-generate output and CSV, so purposes are separated by something else than comma, so we can regexp-parse the malformed JSON output easier.

In [55]:
import re
for i, row in relevant.iterrows():
    #preprocessing...
    relevant.at[i, 'refs'] = '{"refs": '+re.sub("(\w+):\s'?([\w*\-.;:\w*]*)'?", r'"\1": "\2"', str(row.refs))+'}'
    relevant.at[i, 'tags'] = '{"tags": '+re.sub("(\w+):\s'?([\w*\-.;:\w*]*)'?", r'"\1": "\2"', str(row.tags))+'}'
relevant['tags'].head(5)[1]

'{"tags": [{"key": "categories-storage", "value_type": "string", "value_string": "00", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "storage-ttl", "value_type": "string", "value_string": "01", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "recipients", "value_type": "string", "value_string": "", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "purposes", "value_type": "string", "value_string": "00;01;02", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "legal-basis", "value_type": "string", "value_string": "01;01;02", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "categories-processing", "value_type": "string", "value_string": "00", "value_bool": "False", "value_long": "0", "value_double": "0", "value_binary": ""}, {"key": "automation", "value_type": "string", "v

In [80]:
import json
## traverse dict a and merge dict b

def load_dict(val):
    #TODO: later remove artificially added key in preprocessing, so json.loads gets us a list of objects directly
    dict_raw = json.loads(val)
    #flatten dict
    listOfObjects = dict_raw['tags']
    new = []
    for obj in listOfObjects:
        newObj = {
            obj['key']: obj['value_string']
        }
        new.append(newObj)
    return new

def deep_merge(a, b):
    b_as_dict = json.loads(b)
    a.update(b_as_dict)

In [81]:
for i, row in relevant.head(10).iterrows():
    objects = load_dict(row.tags)
    print(objects)

[{'internal.span.format': 'proto'}]
[{'categories-storage': '00'}, {'storage-ttl': '01'}, {'recipients': ''}, {'purposes': '00;01;02'}, {'legal-basis': '01;01;02'}, {'categories-processing': '00'}, {'automation': ''}, {'productIds': 'rt2'}, {'legitimate-interest': '01;02'}, {'internal.span.format': 'proto'}]
[{'legal-basis': '00:00;01:00'}, {'recipients': 'none'}, {'productIds': 'rt2'}, {'purposes': '00;01'}, {'categories-processing': '01;03'}, {'categories-storage': '03'}, {'storage-ttl': '00'}, {'sources': 'none'}, {'automation': 'none'}, {'internal.span.format': 'proto'}]
[{'internal.span.format': 'proto'}]
[{'automation': ''}, {'purposes': '00'}, {'legal-basis': '00'}, {'legitimate-interest': ''}, {'categories-processing': ''}, {'categories-storage': '01'}, {'storage-ttl': '00'}, {'recipients': ''}, {'internal.span.format': 'proto'}]
[{'internal.span.format': 'proto'}]
[{'sampler.type': 'probabilistic'}, {'sampler.param': ''}, {'purposes': '00'}, {'legal-basis': '00:00'}, {'categor

In [56]:
def merge(a, b):
    b_as_dict = json.loads(b)
    a.update(b_as_dict)
    
trace_id_old = ''
tagset = {}
tagsets = []
for i, row in relevant.iterrows():
    if row.trace_id == trace_id_old:
        merge(tagset, row.tags)
    else:
        tagsets.append({trace_id_old: tagset})
        tagset = json.loads(row.tags)
        trace_id_old = row.trace_id
tagsets

[{'': {}},
 {'0x00000000000000000006466589aed53f': {'tags': [{'key': 'sampler.type',
     'value_type': 'string',
     'value_string': 'probabilistic',
     'value_bool': 'False',
     'value_long': '0',
     'value_double': '0',
     'value_binary': ''},
    {'key': 'sampler.param',
     'value_type': 'float64',
     'value_string': '',
     'value_bool': 'False',
     'value_long': '0',
     'value_double': '0.1',
     'value_binary': ''},
    {'key': 'purposes',
     'value_type': 'string',
     'value_string': '00',
     'value_bool': 'False',
     'value_long': '0',
     'value_double': '0',
     'value_binary': ''},
    {'key': 'legal-basis',
     'value_type': 'string',
     'value_string': '00:00',
     'value_bool': 'False',
     'value_long': '0',
     'value_double': '0',
     'value_binary': ''},
    {'key': 'categories-processing',
     'value_type': 'string',
     'value_string': '00;02;03',
     'value_bool': 'False',
     'value_long': '0',
     'value_double': '0',
   